In [1]:
pip install --upgrade transformers==4.30.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [99]:
import pandas as pd
import datetime as dt
import numpy as np
import os
def change_dolar(local_value:float,
    change_rate:float):
  return local_value*change_rate




path_change=r"/content/drive/MyDrive/Hack Corruption - Contractor/datos/Tasa_cambio_Colombia.csv"
path_data=r"/content/drive/MyDrive/Hack Corruption - Contractor/datos/Colombia/main.csv"
path_CPI=r"/content/drive/MyDrive/Hack Corruption - Contractor/datos/USA_CPI.xlsx"
path_data_gener=r"/content/drive/MyDrive/Hack Corruption - Contractor/datos/Colombia/"

n=0

for path in os.listdir(path_data_gener):


  #Loading the exchange rate for USD standaricing
  exchange_rate=pd.read_csv(path_change)
  exchange_rate["exchange_rate"]=4500


  #Loading the dataset  and cleaning dates
  records=pd.read_csv(path_data_gener+path,skiprows=range(1,250000))


  records["Fecha"]=records["date"].apply(
      lambda x:dt.datetime.strptime(x[:10],"%Y-%m-%d").year
  )

  #Loading CPI for real value updating
  CPI=pd.read_excel(path_CPI)
  CPI=CPI[["Year","Avg"]].dropna()
  last_year=CPI["Avg"][len(CPI)-1]

  #merging tables
  records["exchange_rate"]=4500

  #We use the last inflation data and normalize everything to the last year
  records=records.merge(CPI,how="left",left_on=["Fecha"],right_on=["Year"])
  records["Avg"]=records["Avg"].fillna(last_year).apply(lambda x:x/last_year)

  #We scale every value in million dolars
  records["value_million_dolar"]=records.apply(lambda row:row["tender_value_amount"]/(row["exchange_rate"]*row["Avg"]*1e3)
                                      if row["tender_value_currency"]=="COP" else row["tender_value_amount"]/(row["Avg"]*1e3),axis=1)

  if n==0:
    joined=records
  else:
    joined=joined.append(records)


joined=joined.dropna(subset=["tender_description","value_million_dolar"])
joined=joined[joined["value_million_dolar"]<=10000]

<ipython-input-99-f0274ffb155b>:28: DtypeWarning: Columns (12,23,27,29,35,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  records=pd.read_csv(path_data_gener+path,skiprows=range(1,250000))


In [100]:
joined

,_link,id,mes,tag,uri,anio,date,ocid,initiationType,plataforma,...,fechaActualizacion_$date,tender_awardPeriod_endDate,planning_budget_projectID,tender_awardAddress_streetAddress,tender_enquiryPeriod_endDate,Fecha,exchange_rate,Year,Avg,value_million_dolar
0,249999,ocds-k50g02-22-4-12906365-2022-02-14T00:00:00....,2,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-02-14T00:00:00.000Z,ocds-k50g02-22-4-12906365,tender,SECOPI,...,1650509592085,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,2.777778
1,250000,ocds-k50g02-22-4-12906338-2022-02-14T00:00:00....,2,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-02-14T00:00:00.000Z,ocds-k50g02-22-4-12906338,tender,SECOPI,...,1650509591605,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,0.582222
2,250001,ocds-k50g02-22-4-12906370-2022-02-14T00:00:00....,2,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-02-14T00:00:00.000Z,ocds-k50g02-22-4-12906370,tender,SECOPI,...,1650509592217,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,6.665556
3,250002,ocds-k50g02-22-4-12906353-2022-02-14T00:00:00....,2,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-02-14T00:00:00.000Z,ocds-k50g02-22-4-12906353,tender,SECOPI,...,1650509591737,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,1.066667
4,250003,ocds-k50g02-22-4-12906397-2022-02-14T00:00:00....,2,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-02-14T00:00:00.000Z,ocds-k50g02-22-4-12906397,tender,SECOPI,...,1650509592911,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,0.922911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67086,317085,ocds-k50g02-22-4-12676384-2022-01-07T00:00:00....,1,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-01-07T00:00:00.000Z,ocds-k50g02-22-4-12676384,tender,SECOPI,...,1650503297184,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,2.652906
67087,317086,ocds-k50g02-22-4-12676383-2022-01-07T00:00:00....,1,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-01-07T00:00:00.000Z,ocds-k50g02-22-4-12676383,tender,SECOPI,...,1643182006030,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,0.055556
67088,317087,ocds-k50g02-22-4-12676389-2022-01-07T00:00:00....,1,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-01-07T00:00:00.000Z,ocds-k50g02-22-4-12676389,tender,SECOPI,...,1650503297226,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,0.200000
67089,317088,ocds-k50g02-22-12-12698033-2022-01-14T00:00:00...,1,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-01-14T00:00:00.000Z,ocds-k50g02-22-12-12698033,tender,SECOPI,...,1643180164926,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,2.666667


In [101]:
#Declaramos las variables que vamos a usar en predicción



variables_y=["value_million_dolar"]

variables_cat=['tender_status']
variables_reg=["mes"]
variables_text=["tender_description"]


In [102]:
# -*- coding: utf-8 -*-
"""
Created on Sun Dec 10 15:13:07 2023

@author: usuario
"""

import pandas as pd

import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim

from tqdm import tqdm

casa=joined[variables_text+variables_y]
casa[variables_y]=casa[variables_y].astype(int)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class BertDataset(Dataset):
    def __init__(self, tokenizer,max_length):
        super(BertDataset, self).__init__()
        self.train_csv=casa
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,0]

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long).to(device),
            'mask': torch.tensor(mask, dtype=torch.long).to(device),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long).to(device),
            'target': torch.tensor(self.train_csv.iloc[index, 1], dtype=torch.int).to(device)
            }
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-multilingual-cased")

dataset= BertDataset(tokenizer, max_length=100)
batch_size=64
dataloader=DataLoader(dataset=dataset,batch_size=batch_size)

class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.bert_model = transformers.BertModel.from_pretrained("bert-base-multilingual-cased")
        self.medium2 = nn.Linear(768, 15000)
        self.medium3 = nn.ReLU()
        self.medium4=nn.Linear(15000,15000,bias=True)
        self.medium5 = nn.Sigmoid()
        self.medium6=nn.Linear(15000,1500,bias=True)
        self.medium7 = nn.Sigmoid()
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(1500, 1,bias=True)
        self.l1_strength=0.00003

    def forward(self,ids,mask,token_type_ids):
        _,o2= self.bert_model(ids,attention_mask=mask,token_type_ids=token_type_ids, return_dict=False)
        medium2=self.medium2(o2)
        medium3=self.medium3(medium2)
        medium4=self.medium4(medium3)
        medium5=self.medium5(medium4)
        medium6=self.medium5(medium4)
        medium7=self.medium6(medium5)
        medium8=self.drop(medium7)
        out= self.out(medium8)
        return medium7

    def l1_regularization(self):
        n=0
        l1_loss_example = 0
        #the next structure is to only recognize the layers that are finetuned
        #this makes the exercise faster and allows to update the strength easier
        for param in self.parameters():
            n+=1
            if n>197:
                l1_loss_example += torch.sum(torch.abs(param))
        return self.l1_strength * (l1_loss_example)

model=BERT()





def predict(epochs,dataloader,model,loss_fn,optimizer):
    num=1
    acc=0
    loop=tqdm(enumerate(dataloader),leave=False,total=len(dataloader))
    results=[]
    for batch, dl in loop:
      ids=dl['ids']
      token_type_ids=dl['token_type_ids']
      mask= dl['mask']
      label=dl['target']
      label = label.unsqueeze(1)

      optimizer.zero_grad()

      output=model(
          ids=ids,
          mask=mask,
          token_type_ids=token_type_ids)
      label = label.type_as(output)
      for i in output.cpu().detach().numpy():
          results.append(i)


    print(acc/num)


    return results



model.load_state_dict(torch.load(r"/content/drive/MyDrive/Hack Corruption - Contractor/modelos/primero.pt"))



model.to(device)

loss_fn = nn.L1Loss()

#Initialize Optimizer
optimizer= optim.Adam(model.parameters(),lr= 1e-5)


<ipython-input-102-a3703c086873>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  casa[variables_y]=casa[variables_y].astype(int)
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if yo

In [103]:
loss_fn = nn.L1Loss()

#Initialize Optimizer
optimizer= optim.Adam(model.parameters(),lr= 1e-5)

predicted=pd.DataFrame(predict(100000, dataloader, model, loss_fn, optimizer))

  0%|          | 0/1048 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0.0


In [104]:
data_categ=predicted.merge(pd.get_dummies(joined[variables_cat].reset_index(drop=True).astype("str")),left_index=True, right_index=True)

In [ ]:
import pandas as pd


from sklearn.metrics import r2_score
import json

import pickle
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error

#this will be later transplanted to the main file, but for now I need to test
#the code somewhere and tdidn´t find a better place
#some paths to where things are


#this cant be change, they are how rn where train (will be changed manually)



#takes only the regresive variables


data_value=joined[variables_y]



data_categ_train, data_categ_test, data_value_train, data_value_test = train_test_split(
     data_categ, data_value, test_size=0.15, random_state=1,shuffle=True)

dtrain_reg = xgb.DMatrix(data_categ_train, data_value_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(data_categ_test, data_value_test, enable_categorical=True)
pure_data = xgb.DMatrix(data_categ, data_value, enable_categorical=True)


if True:
    n = 20000
    params = {"objective": "reg:pseudohubererror","reg_alpha":10,"reg_lambda":10
              ,"rate_drop":0.1}
    evals = [(dtrain_reg, "train"),(dtest_reg, "validation") ]
    reg = xgb.train(
       params=params,
       dtrain=dtrain_reg,
       num_boost_round=n,
       evals=evals,
       verbose_eval=25,
       early_stopping_rounds=30
       )
    pickle.dump(reg, open(r'/content/drive/MyDrive/Hack Corruption - Contractor/modelos/modelxgboostcol.pkl','wb'))
else:
    with open(r"/content/drive/MyDrive/Hack Corruption - Contractor/modelos/modelxgboostcol.pkl", "rb") as input_file:
      reg = pickle.load(input_file)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [17:14:37] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "rate_drop" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mphe:14.85875	validation-mphe:14.23362
[25]	train-mphe:14.64238	validation-mphe:14.08488
[50]	train-mphe:14.55520	validation-mphe:14.05351
[75]	train-mphe:14.48271	validation-mphe:14.03552
[100]	train-mphe:14.42811	validation-mphe:14.02323
[125]	train-mphe:14.38794	validation-mphe:14.01425
[150]	train-mphe:14.34387	validation-mphe:14.00628
[175]	train-mphe:14.29989	validation-mphe:13.99758
[200]	train-mphe:14.26038	validation-mphe:13.99048
[225]	train-mphe:14.22438	validation-mphe:13.98518
[250]	train-mphe:14.19175	validation-mphe:13.98036
[275]	train-mphe:14.16097	validation-mphe:13.97528
[300]	train-mphe:14.13238	validation-mphe:13.97082
[325]	train-mphe:14.10371	validation-mphe:13.96853
[350]	train-mphe:14.08049	validation-mphe:13.96751
[375]	train-mphe:14.05697	validation-mphe:13.96530
[400]	train-mphe:14.03211	validation-mphe:13.96169
[425]	train-mphe:14.00883	validation-mphe:13.96000
[450]	train-mphe:13.98595	validation-mphe:13.95650
[475]	train-mphe:13.96525	validation

In [ ]:



predict=reg.predict(pure_data)


joined["predict"]=predict

In [ ]:
joined

,_link,id,mes,tag,uri,anio,date,ocid,initiationType,plataforma,...,tender_awardPeriod_endDate,planning_budget_projectID,tender_awardAddress_streetAddress,tender_enquiryPeriod_endDate,Fecha,exchange_rate,Year,Avg,value_million_dolar,predict
0,249999,ocds-k50g02-22-4-12906365-2022-02-14T00:00:00....,2,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-02-14T00:00:00.000Z,ocds-k50g02-22-4-12906365,tender,SECOPI,...,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,2.777778,3.019584
1,250000,ocds-k50g02-22-4-12906338-2022-02-14T00:00:00....,2,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-02-14T00:00:00.000Z,ocds-k50g02-22-4-12906338,tender,SECOPI,...,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,0.582222,0.511309
2,250001,ocds-k50g02-22-4-12906370-2022-02-14T00:00:00....,2,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-02-14T00:00:00.000Z,ocds-k50g02-22-4-12906370,tender,SECOPI,...,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,6.665556,6.865267
3,250002,ocds-k50g02-22-4-12906353-2022-02-14T00:00:00....,2,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-02-14T00:00:00.000Z,ocds-k50g02-22-4-12906353,tender,SECOPI,...,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,1.066667,1.534113
4,250003,ocds-k50g02-22-4-12906397-2022-02-14T00:00:00....,2,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-02-14T00:00:00.000Z,ocds-k50g02-22-4-12906397,tender,SECOPI,...,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,0.922911,0.863326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67086,317085,ocds-k50g02-22-4-12676384-2022-01-07T00:00:00....,1,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-01-07T00:00:00.000Z,ocds-k50g02-22-4-12676384,tender,SECOPI,...,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,2.652906,0.380447
67087,317086,ocds-k50g02-22-4-12676383-2022-01-07T00:00:00....,1,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-01-07T00:00:00.000Z,ocds-k50g02-22-4-12676383,tender,SECOPI,...,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,0.055556,2.776282
67088,317087,ocds-k50g02-22-4-12676389-2022-01-07T00:00:00....,1,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-01-07T00:00:00.000Z,ocds-k50g02-22-4-12676389,tender,SECOPI,...,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,0.200000,0.002828
67089,317088,ocds-k50g02-22-12-12698033-2022-01-14T00:00:00...,1,compiled,https://www.contratos.gov.co/consultas/detalle...,2022,2022-01-14T00:00:00.000Z,ocds-k50g02-22-12-12698033,tender,SECOPI,...,NaN,NaN,NaN,NaN,2022,4500,2022,1.0,2.666667,2.933180


In [ ]:

import pandas as pd
from scipy.stats import norm
from scipy import stats

data1=joined

size=2

data1["range-"]=data1["predict"].apply(lambda x:x*(1-size))
data1["range+"]=data1["predict"].apply(lambda x:x*(1+size))

data1["in_range"]=data1.apply(
    lambda row:1 if row["value_million_dolar"]<row["range+"] and row["value_million_dolar"]>row["range-"] else 0,
    axis=1)

data1["in_range"].mean()

data1["perc_error"]=data1.apply(
    lambda row:(row["value_million_dolar"]-row["predict"])/row["predict"],
    axis=1)

num_contracts=data1["perc_error"].count()

num_contracts

contr_inrange=data1["perc_error"][data1["in_range"]==1].count()
print(contr_inrange/num_contracts)



mu,sigma,kurt=stats.t.fit(data1["perc_error"][data1["in_range"]==1])

mu

data1["perc_error"][data1["in_range"]==1].hist(bins=100,density=True)

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

n, bins, patches = plt.hist(
    data1["perc_error"][data1["in_range"]==1],bins=100,density=True
    )

# add a 'best fit' line
y = stats.t.pdf( bins, mu, sigma,kurt)
l = plt.plot(bins, y, 'r--', linewidth=2)

distribution=stats.t( mu, sigma,kurt)

distribution

help(distribution.pdf)

mu

distribution.pdf(-0.151242)

for i in range(0,100):
  print(distribution.pdf(i/100-0.5))

data1["likelihood"]=data1["perc_error"].apply(lambda x: distribution.pdf(x))

data1["likelihood"].hist(bins=1000)



In [ ]:
#data1.to_excel(r"/content/drive/MyDrive/Hack Corruption - Contractor/datos/resultados finales/resultadoscol.xlsx")

In [ ]:
data1[["value_million_dolar","predict"]]

,value_million_dolar,predict
0,2.777778,3.019584
1,0.582222,0.511309
2,6.665556,6.865267
3,1.066667,1.534113
4,0.922911,0.863326
...,...,...
67086,2.652906,0.380447
67087,0.055556,2.776282
67088,0.200000,0.002828
67089,2.666667,2.933180


In [ ]:
subset=data1[data1["predict"]<=30]
subset=subset[subset["value_million_dolar"]<=30]

In [ ]:
print(data1.count()[0])
print(data1[data1["likelihood"]<0.05].count()[0])

In [ ]:
data1[data1["likelihood"]<0.01]

In [ ]:
for i in subset[["value_million_dolar","predict","likelihood"]].iloc()[0]:
  print(i)

In [ ]:
import seaborn
seaborn.histplot(subset[["predict","value_million_dolar"]],x="predict",y="value_million_dolar",cbar=True,bins=100)

In [ ]:
plt.scatter(subset["value_million_dolar"],subset["predict"])
z = np.polyfit(subset["value_million_dolar"], subset["predict"], 1)
p = np.poly1d(z)
plt.plot(subset["value_million_dolar"],p(subset["value_million_dolar"]),"r--")